In [44]:
import cv2 as cv
import numpy as np 
import matplotlib.pyplot as plt

img = cv.imread('Data Set\Answer Scripts\Answer Scripts\A\JJ502.jpg', cv.IMREAD_GRAYSCALE)
assert img is not None

scale_percent_w= 25  # Percentage of the original size width
scale_percent_h= 30  # Percentage of the original size height
width = int(img.shape[1] * scale_percent_w / 100)
height = int(img.shape[0] * scale_percent_h/ 100)
dim = (width, height)

# Resize image
resized_image = cv.resize(img, dim, interpolation=cv.INTER_AREA)

# size of the image
print(resized_image.shape)

imshow = cv.imshow('Image', resized_image)
cv.waitKey(0)
cv.destroyAllWindows()


(639, 772)


In [49]:
#extracting the  area of interest
x, y, w, h = 10, 180, 550, 350
roi = resized_image[y:y+h, x:x+w]

# Display the extracted region
cv.imshow('ROI', roi)
cv.waitKey(0)
cv.destroyAllWindows()

In [6]:
import cv2
import numpy as np

def extract_student_answers(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize the image if it's too large (optional step)
    height, width = img.shape
    if height > 1000:
        img = cv2.resize(img, (width // 2, height // 2))
    
    # Apply Gaussian blur to smooth the image (helpful for better circle detection)
    blurred = cv2.GaussianBlur(img, (9, 9), 3.7)

    # Apply Hough Circle Transform to detect circles (bubbles)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                               param1=50, param2=30, minRadius=9, maxRadius=19)

    # If no circles are found, return an empty list
    if circles is None:
        print("No bubbles found!")
        return []

    # Convert circle coordinates to integers
    circles = np.round(circles[0, :]).astype("int")

    # Visualize the detected circles on the image for debugging
    output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for (x, y, r) in circles:
        cv2.circle(output, (x, y), r, (0, 255, 0), 2)

    # Show the image with detected bubbles
    cv2.imshow("Detected Bubbles", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Sort the circles by their y-coordinate, and then x-coordinate (top-to-bottom, left-to-right)
    circles = sorted(circles, key=lambda c: (c[1], c[0]))

    # Define the number of questions and number of choices per question
    num_questions = 50
    num_choices = 5  # Assuming 5 choices per question (1-5)

    # Initialize a list to store detected answers
    student_answers = []

    # Group circles into batches of 5 (one batch per question)
    for q in range(num_questions):
        question_bubbles = circles[q * num_choices:(q + 1) * num_choices]
        
        # Sort the bubbles horizontally to map them to the correct answer options (1-5)
        question_bubbles = sorted(question_bubbles, key=lambda c: c[0])  # Sort left-to-right

        filled_option = None
        max_intensity = 0
        
        # Check which bubble is filled (darkest bubble)
        for idx, (x, y, r) in enumerate(question_bubbles):
            # Create a circular mask to isolate the bubble
            mask = np.zeros(img.shape, dtype="uint8")
            cv2.circle(mask, (x, y), r, 255, -1)

            # Use the mask to count the non-zero pixels inside the bubble
            bubble_intensity = cv2.mean(img, mask=mask)[0]

            # Assume the darkest bubble is the filled one (least intensity value)
            if bubble_intensity < max_intensity or filled_option is None:
                max_intensity = bubble_intensity
                filled_option = idx + 1  # Option is 1-indexed

        # Append the detected answer to the list
        student_answers.append(filled_option)

    return student_answers

# Example usage:
image_path = 'Data Set\Answer Scripts\Answer Scripts\A\JJ502.jpg'
student_answers = extract_student_answers(image_path)
print(student_answers)


[1, 1, 1, 1, 2, 1, 1, 1, 2, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [3]:
import cv2
import numpy as np

def detect_filled_answers(image_path):
    # Read the grayscale image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian blur to smooth the image
    blurred = cv2.GaussianBlur(img, (9, 9), 3.7)

    # Apply Hough Circle Transform to detect circles (bubbles)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                               param1=50, param2=30, minRadius=9, maxRadius=19)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")

        # Sort circles by their y-coordinate to group them into rows
        circles = sorted(circles, key=lambda x: (x[1], x[0]))

        # Assuming 50 questions and each question has 5 answer options
        num_choices = 5
        num_questions = len(circles) // num_choices

        # Initialize a list to store the selected answers
        student_answers = []

        # Iterate over each question's circles (grouped by rows)
        for q in range(num_questions):
            # Get the circles for this question (5 circles per question)
            question_circles = circles[q * num_choices:(q + 1) * num_choices]

            # Sort the circles from left to right (corresponding to answer options 1-5)
            question_circles = sorted(question_circles, key=lambda x: x[0])

            filled_option = None
            min_intensity = float('inf')

            # Iterate through each option in the question
            for idx, (x, y, r) in enumerate(question_circles):
                # Create a mask for the current circle
                mask = np.zeros_like(img)
                cv2.circle(mask, (x, y), r, 255, -1)

                # Calculate the mean intensity inside the circle (lower means it's filled)
                mean_intensity = cv2.mean(img, mask=mask)[0]

                # Keep track of the lowest intensity (darkest circle, which is filled)
                if mean_intensity < min_intensity:
                    min_intensity = mean_intensity
                    filled_option = idx + 1  # Answer options are 1-indexed

            # Append the selected answer for this question
            student_answers.append(filled_option)

    return student_answers


# Example usage:
image_path = 'Data Set\Answer Scripts\Answer Scripts\A\JJ502.jpg'  # Path to the image you provided
student_answers = detect_filled_answers(image_path)

# Print the answers detected
for q, answer in enumerate(student_answers, 1):
    print(f"Question {q}: Answer {answer}")


Question 1: Answer 1
Question 2: Answer 1
Question 3: Answer 2
Question 4: Answer 2
Question 5: Answer 1


In [6]:
import cv2
import numpy as np

def extract_answer_area(image_path, output_size=(700, 500)):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to smooth the image
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    
    # Apply edge detection (Canny)
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour which should correspond to the answer sheet area
    largest_contour = max(contours, key=cv2.contourArea)

    # Approximate the contour to get a quadrilateral (4 corners)
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx_corners = cv2.approxPolyDP(largest_contour, epsilon, True)

    if len(approx_corners) != 4:
        print("Unable to detect 4 corners. Detected corners:", len(approx_corners))
        return None

    # Get the 4 corners of the approximated contour
    approx_corners = approx_corners.reshape(4, 2)
    
    # Sort the corners to match the order: [top-left, top-right, bottom-right, bottom-left]
    rect = np.zeros((4, 2), dtype="float32")

    s = approx_corners.sum(axis=1)
    rect[0] = approx_corners[np.argmin(s)]  # Top-left
    rect[2] = approx_corners[np.argmax(s)]  # Bottom-right

    diff = np.diff(approx_corners, axis=1)
    rect[1] = approx_corners[np.argmin(diff)]  # Top-right
    rect[3] = approx_corners[np.argmax(diff)]  # Bottom-left

    # Define the desired output size (warped size)
    (width, height) = output_size

    # Set the destination points for perspective transform (a rectangle of given size)
    dst = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]], dtype="float32")

    # Compute the perspective transform matrix
    M = cv2.getPerspectiveTransform(rect, dst)

    # Apply the warp perspective transformation
    warped = cv2.warpPerspective(img, M, (width, height))

    return warped

# Example usage:
image_path = 'Data Set\Answer Scripts\Answer Scripts\A\JJ503.jpg'  # Path to your image
output_size = (700, 500)  # You can adjust the size as needed

warped_image = extract_answer_area(image_path, output_size)

# Save the warped image or display it
if warped_image is not None:
    cv2.imshow("Warped Answer Sheet", warped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite('warped_answer_sheet.png', warped_image)


In [22]:
import cv2
import numpy as np

def detect_bubbles_hough(warped_image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # Apply adaptive threshold to highlight circles
    _, thresh = cv2.threshold(blurred, 150, 225, cv2.THRESH_BINARY_INV)

    # Detect circles using Hough Circle Transform
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                               param1=50, param2=30, minRadius=5, maxRadius=15)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")

        # Loop over the detected circles and draw them
        for (x, y, r) in circles:
            cv2.circle(warped_image, (x, y), r, (0, 255, 0), 2)  # Draw green circle
            cv2.circle(warped_image, (x, y), 2, (0, 0, 255), 3)  # Draw red center dot

    return warped_image, circles

# Example usage
# Load the provided warped image
#warped_image = cv2.imread('Data Set\Answer Scripts\Answer Scripts\A\JJ503.jpg')
warped_image = cv2.imread('warped_answer_sheet.png')

# Call the function to detect and circle the bubbles using Hough Circles
detected_image, bubbles = detect_bubbles_hough(warped_image)

# Display the image with the detected bubbles
cv2.imshow("Detected Bubbles", detected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the output image with detected bubbles
cv2.imwrite('bubbles_detected_hough.png', detected_image)

# Optionally, print detected bubbles (if any) with their coordinates and radius
if bubbles is not None:
    print(f"Detected {len(bubbles)} bubbles.")
    for i, (x, y, r) in enumerate(bubbles):
        print(f"Bubble {i+1}: Center (x: {x}, y: {y}), Radius: {r}")
else:
    print("No bubbles detected.")


Detected 21 bubbles.
Bubble 1: Center (x: 393, y: 128), Radius: 20
Bubble 2: Center (x: 466, y: 472), Radius: 21
Bubble 3: Center (x: 41, y: 76), Radius: 16
Bubble 4: Center (x: 341, y: 422), Radius: 16
Bubble 5: Center (x: 533, y: 79), Radius: 18
Bubble 6: Center (x: 170, y: 373), Radius: 16
Bubble 7: Center (x: 193, y: 470), Radius: 15
Bubble 8: Center (x: 352, y: 28), Radius: 15
Bubble 9: Center (x: 325, y: 176), Radius: 17
Bubble 10: Center (x: 45, y: 27), Radius: 19
Bubble 11: Center (x: 229, y: 274), Radius: 18
Bubble 12: Center (x: 533, y: 471), Radius: 15
Bubble 13: Center (x: 649, y: 74), Radius: 20
Bubble 14: Center (x: 395, y: 28), Radius: 14
Bubble 15: Center (x: 605, y: 27), Radius: 19
Bubble 16: Center (x: 485, y: 374), Radius: 16
Bubble 17: Center (x: 665, y: 27), Radius: 13
Bubble 18: Center (x: 667, y: 473), Radius: 11
Bubble 19: Center (x: 673, y: 128), Radius: 18
Bubble 20: Center (x: 483, y: 224), Radius: 15
Bubble 21: Center (x: 532, y: 128), Radius: 11
